In [1]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetV2S
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.utils import class_weight
import numpy as np
import os



In [2]:
dataset_path = r"C:\Users\AI Server1\Desktop\Osteoarthritis_sehar\new_dataset2"  #0,2,4 classes(healthy,minimal,severe)
img_size = (300, 300)
batch_size = 8

In [3]:
train_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(dataset_path, "train"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(dataset_path, "val"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_gen = ImageDataGenerator(rescale=1./255).flow_from_directory(
    os.path.join(dataset_path, "test"),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

Found 3975 images belonging to 3 classes.
Found 567 images belonging to 3 classes.
Found 1137 images belonging to 3 classes.


In [4]:
class_labels = np.unique(train_gen.classes)
class_weights_dict = dict(zip(class_labels, class_weight.compute_class_weight(class_weight='balanced', classes=class_labels, y=train_gen.classes)))


In [5]:
base_model = EfficientNetV2S(weights='imagenet', include_top=False, input_shape=(300, 300, 3))
base_model.trainable = False 

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.2)(x)
output = Dense(train_gen.num_classes, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)


model.compile(optimizer=Adam(1e-4), loss='categorical_crossentropy', metrics=['accuracy'])


callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
    EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
]

In [6]:
print(" Training top classifier head...")
model.fit(train_gen, validation_data=val_gen, epochs=10, class_weight=class_weights_dict, callbacks=callbacks)

base_model.trainable = True
model.compile(optimizer=Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])

print("Fine-tuning model : ")
model.fit(train_gen, validation_data=val_gen, epochs=20, class_weight=class_weights_dict, callbacks=callbacks)

 Training top classifier head...
Epoch 1/10
497/497 [==============================] - 360s 709ms/step - loss: 1.1020 - accuracy: 0.3550 - val_loss: 1.1000 - val_accuracy: 0.1675 - lr: 1.0000e-04
Epoch 2/10
497/497 [==============================] - 332s 668ms/step - loss: 1.0988 - accuracy: 0.2639 - val_loss: 1.1051 - val_accuracy: 0.1675 - lr: 1.0000e-04
Epoch 3/10
497/497 [==============================] - 364s 733ms/step - loss: 1.0968 - accuracy: 0.2891 - val_loss: 1.1390 - val_accuracy: 0.1076 - lr: 1.0000e-04
Epoch 4/10
497/497 [==============================] - 359s 723ms/step - loss: 1.0939 - accuracy: 0.2709 - val_loss: 1.0669 - val_accuracy: 0.5767 - lr: 1.0000e-04
Epoch 5/10
497/497 [==============================] - 339s 682ms/step - loss: 1.0907 - accuracy: 0.3660 - val_loss: 1.0812 - val_accuracy: 0.2011 - lr: 1.0000e-04
Epoch 6/10
497/497 [==============================] - 339s 682ms/step - loss: 1.0961 - accuracy: 0.2631 - val_loss: 1.1848 - val_accuracy: 0.1041 - lr: 

In [7]:
model.save("EfficientNetV2S.h5")

In [8]:
loss, acc = model.evaluate(test_gen)
print(f"Final Test Accuracy: {acc:.4f}, Loss: {loss:.4f}")

143/143 [==============================] - 79s 553ms/step - loss: 0.3738 - accuracy: 0.8461
Final Test Accuracy: 0.8461, Loss: 0.3738
